In [ ]:
# pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.4 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
import plotly.graph_objects as go
import plotly.io as pio

# Set Plotly to work in Google Colab
pio.renderers.default = 'colab'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tif_data = pd.read_csv('/content/drive/MyDrive/480-497-Demographic TIF Team/Transfers/DataFolder/Yates_Data/Master_TIF_formatted.csv')
tif_bounds = gpd.read_file('/content/drive/MyDrive/480-497-Demographic TIF Team/Transfers/DataFolder/Boundaries - Tax Increment Financing Districts.geojson')

In [ ]:
tif_transfers_dict = {}
for index, row in tif_data[tif_data['tif_year'] == 2022].iterrows():
    tif_transfers_dict[row['tif_name']] = (int(row['cumulative_transfers_in']) - int(row['cumulative_transfers_out'])) / (int(row['cumulative_property_tax_extraction'])) if int(row['cumulative_property_tax_extraction']) != 0 else 0

tif_transfers = pd.DataFrame([(k,v) for k,v in tif_transfers_dict.items()], columns=['tif_name', 'Amount'])
tif_transfers = tif_transfers[tif_transfers["Amount"] != 0]
tif_bounds.rename(columns={"name":"tif_name"}, inplace=True)
merged = tif_bounds.merge(tif_transfers, on='tif_name', how='left')
merged

,sbif,tif_name,shape_area,show,objectid_1,name_trim,wards_2023,ref,approval_d,objectid,...,comm_area,objectid_2,use,repealed_d,type,shape_le_1,ind,expiration,geometry,Amount
0,N,116th/Avenue O,11512032.0441,1,1,116th/Avenue O,10,T-182,10/31/2018,5167,...,"51,52,55",100,Mixed-use: Comm/Indust/Institut/Parks/Open Space,None,Existing,22113.6634385,Mixed-use,12/31/2042,"MULTIPOLYGON (((-87.53995 41.68459, -87.53995 ...",3.401930
1,Y,Bryn Mawr/Broadway,1497351.76204,1,2,Bryn Mawr/Broadway,48,T- 13,11/4/1998,5207,...,77,38,Mixed-use: Residential/Commercial,None,Existing,10146.7645744,Mixed-use,12/31/2032,"MULTIPOLYGON (((-87.66051 41.98451, -87.66037 ...",2.380356
2,N,51st and Lake Park,99810.7553475,1,3,51st and Lake Park,4,T-175,11/15/2012,5185,...,41,19,Mixed-use: Residential/Commercial/Retail,None,Existing,1319.69144432,Mixed-use,12/31/2036,"MULTIPOLYGON (((-87.58854 41.80155, -87.58902 ...",NaN
3,N,Lakefront,1234864.43789,2,4,Lakefront,4,T-119,3/27/2002,5150,...,36,34,Residential,None,Existing,7017.57899731,Residential,12/31/2026,"MULTIPOLYGON (((-87.59624 41.81693, -87.59676 ...",NaN
4,N,Madden/Wells,5460412.60763,2,5,Madden/Wells,4,T-126,11/6/2002,0,...,"35,36,38",129,Residential,None,Existing,12134.5933039,Residential,12/31/2038,"MULTIPOLYGON (((-87.61454 41.82808, -87.61455 ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Y,Lincoln Avenue,7596571.99264,1,120,Lincoln Avenue,"40,50",T- 77,11/3/1999,5242,...,"2,4,13",120,Mixed-use: Residential/Commercial/Institutional,None,Existing,31074.4029002,Mixed-use,12/31/2035,"MULTIPOLYGON (((-87.71047 41.99673, -87.71047 ...",0.406236
117,Y,63rd/Pulaski,9238352.82417,1,121,63rd/Pulaski,"13,23",T- 98,5/17/2000,5243,...,"57,62,64,65",121,Mixed-use: See Plan for Details,None,Existing,56569.2043874,Mixed-use,12/31/2024,"MULTIPOLYGON (((-87.72180 41.76583, -87.72186 ...",-0.836961
118,N,Cortland and Chicago River,7631475.18575,0,122,Cortland and Chicago River,"2,32",T-183,4/10/2019,5246,...,"7,8,22,24",122,Mixed-use: Res/Comm/Instit/Parks/Open Space,None,Existing,17249.9092933,Mixed-use,12/31/2043,"MULTIPOLYGON (((-87.66612 41.92000, -87.66603 ...",6.972079
119,N,Canal/Congress,3048664.30904,1,123,Canal/Congress,"28,34,42",T- 63,11/12/1998,5249,...,28,123,Mixed-use: Residential/Commercial/Institutional,None,Existing,15935.144818,Mixed-use,12/31/2034,"MULTIPOLYGON (((-87.64485 41.88255, -87.64474 ...",-0.143418


# Total Transfered In/Out / Total tax collected

In [ ]:
# Create a base layer for the interactive heatmap for census tracts
fig = px.choropleth_mapbox(
    merged,
    geojson=merged.geometry,
    locations=merged.index,
    color='Amount',  # Color based on population
    color_continuous_scale=["blue", "red"],  # Choose a color scale
    # range_color=[min(tif_transfers['Amount']), max(tif_transfers['Amount'])],  # Set color range explicitly
    range_color=[0, 1],  # Set color range explicitly
    mapbox_style="carto-positron",
    center={"lat": merged.centroid.y.mean(), "lon": merged.centroid.x.mean()},
    zoom=10,
    opacity=0.5,
    labels={'tif_name'},
    hover_name='tif_name',  # Display community area name on hover
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
tif_transfers_dict = {}
for index, row in tif_data[tif_data['tif_year'] == 2022].iterrows():
    if row['cumulative_transfers_in'] > row['cumulative_transfers_out']:
        tif_transfers_dict[row['tif_name']] = 1
    elif row['cumulative_transfers_in'] < row['cumulative_transfers_out']:
        tif_transfers_dict[row['tif_name']] = -1
    else:
        tif_transfers_dict[row['tif_name']] = 0


tif_transfers = pd.DataFrame([(k,v) for k,v in tif_transfers_dict.items()], columns=['tif_name', 'fact_of_transfer'])
tif_bounds.rename(columns={"name":"tif_name"}, inplace=True)
merged = tif_bounds.merge(tif_transfers, on='tif_name', how='left')
merged

,sbif,tif_name,shape_area,show,objectid_1,name_trim,wards_2023,ref,approval_d,objectid,...,comm_area,objectid_2,use,repealed_d,type,shape_le_1,ind,expiration,geometry,fact_of_transfer
0,N,116th/Avenue O,11512032.0441,1,1,116th/Avenue O,10,T-182,10/31/2018,5167,...,"51,52,55",100,Mixed-use: Comm/Indust/Institut/Parks/Open Space,None,Existing,22113.6634385,Mixed-use,12/31/2042,"MULTIPOLYGON (((-87.53995 41.68459, -87.53995 ...",1.0
1,Y,Bryn Mawr/Broadway,1497351.76204,1,2,Bryn Mawr/Broadway,48,T- 13,11/4/1998,5207,...,77,38,Mixed-use: Residential/Commercial,None,Existing,10146.7645744,Mixed-use,12/31/2032,"MULTIPOLYGON (((-87.66051 41.98451, -87.66037 ...",1.0
2,N,51st and Lake Park,99810.7553475,1,3,51st and Lake Park,4,T-175,11/15/2012,5185,...,41,19,Mixed-use: Residential/Commercial/Retail,None,Existing,1319.69144432,Mixed-use,12/31/2036,"MULTIPOLYGON (((-87.58854 41.80155, -87.58902 ...",0.0
3,N,Lakefront,1234864.43789,2,4,Lakefront,4,T-119,3/27/2002,5150,...,36,34,Residential,None,Existing,7017.57899731,Residential,12/31/2026,"MULTIPOLYGON (((-87.59624 41.81693, -87.59676 ...",0.0
4,N,Madden/Wells,5460412.60763,2,5,Madden/Wells,4,T-126,11/6/2002,0,...,"35,36,38",129,Residential,None,Existing,12134.5933039,Residential,12/31/2038,"MULTIPOLYGON (((-87.61454 41.82808, -87.61455 ...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Y,Lincoln Avenue,7596571.99264,1,120,Lincoln Avenue,"40,50",T- 77,11/3/1999,5242,...,"2,4,13",120,Mixed-use: Residential/Commercial/Institutional,None,Existing,31074.4029002,Mixed-use,12/31/2035,"MULTIPOLYGON (((-87.71047 41.99673, -87.71047 ...",1.0
117,Y,63rd/Pulaski,9238352.82417,1,121,63rd/Pulaski,"13,23",T- 98,5/17/2000,5243,...,"57,62,64,65",121,Mixed-use: See Plan for Details,None,Existing,56569.2043874,Mixed-use,12/31/2024,"MULTIPOLYGON (((-87.72180 41.76583, -87.72186 ...",-1.0
118,N,Cortland and Chicago River,7631475.18575,0,122,Cortland and Chicago River,"2,32",T-183,4/10/2019,5246,...,"7,8,22,24",122,Mixed-use: Res/Comm/Instit/Parks/Open Space,None,Existing,17249.9092933,Mixed-use,12/31/2043,"MULTIPOLYGON (((-87.66612 41.92000, -87.66603 ...",1.0
119,N,Canal/Congress,3048664.30904,1,123,Canal/Congress,"28,34,42",T- 63,11/12/1998,5249,...,28,123,Mixed-use: Residential/Commercial/Institutional,None,Existing,15935.144818,Mixed-use,12/31/2034,"MULTIPOLYGON (((-87.64485 41.88255, -87.64474 ...",-1.0


In [ ]:
nulls = merged[merged['fact_of_transfer'].isnull()]
nulls

,sbif,tif_name,shape_area,show,objectid_1,name_trim,wards_2023,ref,approval_d,objectid,...,comm_area,objectid_2,use,repealed_d,type,shape_le_1,ind,expiration,geometry,fact_of_transfer
6,Y,Stony Island Avenue Commercial and Burnside In...,26720795.1696,1,7,Stony Island Avenue Commercial and Burnside In...,8,T- 54,6/10/1998,5202,...,"44,45,47,48,49,50,51",88,Industrial,None,Existing,93967.9320469,Industrial,12/31/2034,"MULTIPOLYGON (((-87.58533 41.74923, -87.58521 ...",NaN
88,Y,Austin Commercial,11126344.9379,0,92,Austin Commercial,"29,37",T-156,9/27/2007,5199,...,25,87,Commercial,None,Existing,69691.9733985,Commercial,12/31/2031,"MULTIPOLYGON (((-87.74934 41.91005, -87.74933 ...",NaN


In [ ]:
merged.loc[nulls.index, 'fact_of_transfer'] = 0

In [ ]:
merged[merged['fact_of_transfer'].isnull()]

,sbif,tif_name,shape_area,show,objectid_1,name_trim,wards_2023,ref,approval_d,objectid,...,comm_area,objectid_2,use,repealed_d,type,shape_le_1,ind,expiration,geometry,fact_of_transfer


# Discrete map for transfers in > transfers out or vice versa

In [ ]:
fig = px.choropleth_mapbox(
    merged,
    geojson=merged.geometry,
    locations=merged.index,
    color='fact_of_transfer',
    color_discrete_map={
        1: 'red',
        -1: 'blue',
        0: 'green'
    },
    mapbox_style="carto-positron",
    center={"lat": merged.centroid.y.mean(), "lon": merged.centroid.x.mean()},
    zoom=10,
    opacity=0.5,
    labels={'tif_name'},
    hover_name='tif_name',
)
fig.update_layout(
  width=800,  # Set width to 800 pixels
  height=1000  # Set height to 600 pixels
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

# Transfers in - Transfers out by year

In [ ]:
for year in range(2014, 2023):
    tif_transfers_dict = {}
    for index, row in tif_data[tif_data['tif_year'] == year].iterrows():
        if row['tif_name'] in tif_transfers_dict:
            tif_transfers_dict[row['tif_name']] += (int(row['transfers_in']) - int(row['transfers_out']))
        else:
            tif_transfers_dict[row['tif_name']] = (int(row['transfers_in']) - int(row['transfers_out']))

    tif_transfers = pd.DataFrame([(k,v) for k,v in tif_transfers_dict.items()], columns=['tif_name', 'Amount'])
    tif_transfers = tif_transfers[tif_transfers["Amount"] != 0]
    tif_bounds.rename(columns={"name":"tif_name"}, inplace=True)
    merged = tif_bounds.merge(tif_transfers, on='tif_name', how='left')
    # Create a base layer for the interactive heatmap for census tracts
    fig = px.choropleth_mapbox(
        merged,
        geojson=merged.geometry,
        locations=merged.index,
        color='Amount',  # Color based on population
        color_continuous_scale=["blue", "red"],  # Choose a color scale
        # range_color=[min(tif_transfers['Amount']), max(tif_transfers['Amount'])],  # Set color range explicitly
        range_color=[-7000000, 14000000],  # Set color range explicitly
        mapbox_style="carto-positron",
        center={"lat": merged.centroid.y.mean(), "lon": merged.centroid.x.mean()},
        zoom=10,
        opacity=0.5,
        labels={'tif_name'},
        hover_name='tif_name',  # Display community area name on hover
        title=year)

    fig.update_layout(
      width=800,  # Set width to 800 pixels
      height=1000  # Set height to 600 pixels
    )

    fig.show()

Output hidden; open in https://colab.research.google.com to view.